In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import warnings

In [35]:
df = pd.read_csv(r"C:\Users\cex\Downloads\salamanca_idealista.csv")

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Address         1500 non-null   object 
 1   Price           1500 non-null   float64
 2   Rooms           1487 non-null   float64
 3   Squared_meters  1500 non-null   int64  
 4   Floor           1500 non-null   int64  
 5   Link            1500 non-null   object 
 6   Summary         1485 non-null   object 
 7   Type            1500 non-null   object 
 8   Pool            1485 non-null   object 
 9   Furniture       1485 non-null   object 
 10  Exterior        1500 non-null   bool   
 11  Elevator        1500 non-null   bool   
dtypes: bool(2), float64(2), int64(2), object(6)
memory usage: 120.2+ KB


In [37]:
df['Price'] = df['Price']*1000

In [38]:
df

,Address,Price,Rooms,Squared_meters,Floor,Link,Summary,Type,Pool,Furniture,Exterior,Elevator
0,"calle Pollo Martín, 34, Labradores, Salamanca",100000.0,NaN,55,0,www.idealista.com/inmueble/101466881/,ATENCIÓN INMEJORABLE INVERSIÓN! . TOTALMENTE R...,Studio,False,False,True,True
1,"paseo de Canalejas, San Esteban-San Cristobal,...",105000.0,NaN,40,0,www.idealista.com/inmueble/101428581/,"A ESTRENAR! . EN EL PASEO DE CANALEJAS, junto ...",Studio,False,False,True,True
2,"ronda de Sancti-Spiritus, Sancti Spiritus-San ...",110000.0,NaN,33,0,www.idealista.com/inmueble/97782093/,INVERSIÓN GARANTIZADA PARA ALQUILER! . En la C...,Studio,False,False,True,True
3,"plaza Mayor, Centro, Salamanca",220000.0,NaN,38,2,www.idealista.com/inmueble/101377340/,Soho tiene en venta magnifico activo para inve...,Studio,False,False,False,True
4,"calle Oropéndola, 72, Pizarrales, Salamanca",61000.0,NaN,47,1,www.idealista.com/inmueble/1740935/,Precioso estudio ubicado en la zona del carref...,Studio,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,"calle Batalla del Salado, Prosperidad - Delici...",189000.0,3.0,100,4,www.idealista.com/inmueble/101955186/,Estudio Urbano presenta fantástica vivienda en...,Flat,False,False,True,True
1496,"calle Pinto, San Esteban-San Cristobal, Salamanca",295000.0,2.0,85,5,www.idealista.com/inmueble/101981929/,"Magnífico Ático en el Centro de Salamanca, ubi...",Flat,False,False,True,True
1497,"calle Toreses, Rollo - Puente Ladrillo, Salamanca",110000.0,2.0,63,2,www.idealista.com/inmueble/86373366/,"Interesante piso, para entrar a vivir, con Pla...",Flat,False,False,True,True
1498,"calle Larga, 19, San José - Parador - Zurguen,...",124000.0,2.0,78,1,www.idealista.com/inmueble/99282145/,"Vivir en un lugar rodeado de zonas verdes, al ...",Flat,False,False,True,True


## Preparacion de datos

Empezando con la preparacion datos, eliminamos las columnas imnecesarias tales como **Address,Link, y Summary**

In [39]:
df.drop(['Address','Summary','Link'],axis=1,inplace=True)

Despues de eliminar las columnas imnecesarias prosigamos con cambiar las columnas que contienen **Pool,Furniture,Exterior y Elevator.**

In [40]:
df.replace({False: 0, True: 1}, inplace=True)

In [41]:
df

,Price,Rooms,Squared_meters,Floor,Type,Pool,Furniture,Exterior,Elevator
0,100000.0,NaN,55,0,Studio,0.0,0.0,1,1
1,105000.0,NaN,40,0,Studio,0.0,0.0,1,1
2,110000.0,NaN,33,0,Studio,0.0,0.0,1,1
3,220000.0,NaN,38,2,Studio,0.0,0.0,0,1
4,61000.0,NaN,47,1,Studio,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...
1495,189000.0,3.0,100,4,Flat,0.0,0.0,1,1
1496,295000.0,2.0,85,5,Flat,0.0,0.0,1,1
1497,110000.0,2.0,63,2,Flat,0.0,0.0,1,1
1498,124000.0,2.0,78,1,Flat,0.0,0.0,1,1


In [42]:
df['Type'].value_counts()

Flat      1415
Attic       54
Duplex      18
Studio      13
Name: Type, dtype: int64

In [43]:
pd.get_dummies(df["Type"])

,Attic,Duplex,Flat,Studio
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
1495,0,0,1,0
1496,0,0,1,0
1497,0,0,1,0
1498,0,0,1,0


In [44]:
df_dummies = pd.get_dummies(df["Type"])

In [45]:
df['Attic'],df['Duplex'],df['Flat'],df['Studio'] = pd.get_dummies(df["Type"])['Attic'],pd.get_dummies(df["Type"])['Duplex'],pd.get_dummies(df["Type"])['Flat'],pd.get_dummies(df["Type"])['Studio']

In [46]:
df.drop(['Type','Floor'],axis=1,inplace=True)

In [47]:
df

,Price,Rooms,Squared_meters,Pool,Furniture,Exterior,Elevator,Attic,Duplex,Flat,Studio
0,100000.0,NaN,55,0.0,0.0,1,1,0,0,0,1
1,105000.0,NaN,40,0.0,0.0,1,1,0,0,0,1
2,110000.0,NaN,33,0.0,0.0,1,1,0,0,0,1
3,220000.0,NaN,38,0.0,0.0,0,1,0,0,0,1
4,61000.0,NaN,47,0.0,0.0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1495,189000.0,3.0,100,0.0,0.0,1,1,0,0,1,0
1496,295000.0,2.0,85,0.0,0.0,1,1,0,0,1,0
1497,110000.0,2.0,63,0.0,0.0,1,1,0,0,1,0
1498,124000.0,2.0,78,0.0,0.0,1,1,0,0,1,0


**Train Test Split**

In [53]:
X = df.drop(['Price'],axis=1)a
y = df['Price']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 123)

In [54]:
d_train = lgb.Dataset(x_train, label=y_train) # Load the dataset and test

# parameters for this model
params = {
        'n_estimators': 10000,
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'max_depth': -1,
        'learning_rate': 0.01,
        'subsample': 0.72,
        'subsample_freq': 4,
        'feature_fraction': 0.4,
        'lambda_l1': 1,
        'lambda_l2': 1,
        'seed': 46,
        }

clf = lgb.train(params, d_train, 100)

C:\Users\cex\anacondas\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000206 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 125
[LightGBM] [Info] Number of data points in the train set: 1125, number of used features: 7
[LightGBM] [Info] Start training from score 191910.236444
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [55]:
y_pred = clf.predict(x_test)
y_pred

array([221618.43006278, 149454.52752839, 142560.67658596,  92910.82057996,
       433446.04647295, 134857.68259944, 142744.89503689, 132799.52340752,
       117324.26420713, 115496.9974453 , 133325.13860777, 142744.89503689,
       131198.20839718,  97718.98450733, 110898.1814803 , 142744.89503689,
       142560.67658596, 119764.60578292, 163301.98958094, 142960.15680982,
       160287.9052172 , 122689.21141467, 112798.33616243, 115779.21115214,
       133446.71367517, 268433.66327188, 135611.90594015, 433446.04647295,
       187347.84281687, 181614.29068206, 259915.46779683, 157710.1399912 ,
       122689.21141467, 156720.25359928, 122689.21141467, 175380.27736432,
       167266.03742457, 410204.68056741, 187347.84281687, 167605.66130754,
       102596.39799427, 423554.41438857, 267005.48852568, 293387.125398  ,
       174762.17304282, 117324.26420713, 278139.08828221, 388942.35495988,
       161919.60911712, 307681.57224467, 152198.39102486, 295112.77071941,
       120394.59298206, 2

**Evaluando el resultado**

In [56]:
def compute_rmse(model, X, y_true, name):
    y_pred = model.predict(X)
    mse = mean_squared_error(y_true, y_pred)
    rmse = pow(mse,0.5)
    print(f'Root Mean Squared Error for {name}: {rmse}')

In [57]:
compute_rmse(clf, x_train, y_train, 'Training Set')
compute_rmse(clf, x_test, y_test, 'Test Set')

Root Mean Squared Error for Training Set: 56262.26065673925
Root Mean Squared Error for Test Set: 80891.90750749773


El resultado del LGBM está funcionando bastante bien y podríamos usar este modelo en situaciones de la vida real y ayudar a los demás a saber cuánto dinero deben pagar por el apartamento que están buscando.